#  area

In [ ]:

def determine_area(zipcode):
    area = pd.read_csv('data/clean_files/area_FL.csv')
    area.dropna(inplace=True)  
    plans = area[area['zipcode']== zipcode]
    area_key = fred[['IssuerId','ServiceAreaId']]
    area_key.reset_index()
    area_key['IssuerId']=area_key['IssuerId'].astype(int)
    return area_key

area = pd.read_csv('data/clean_files/area_FL.csv')
area.dropna(inplace=True)
area['IssuerId']=area['IssuerId'].astype(int)
area.head()
fred = area[area['zipcode']== 32609]
mark=fred[['IssuerId','ServiceAreaId']]
areas=determine_area(32609)
areas.sort_index()

In [ ]:
area=pd.read_csv('../data/raw_files/Service_Area_PUF.csv',low_memory=False, encoding ='latin1')

area=area[area.DentalOnlyPlan != 'Yes']
area.drop('DentalOnlyPlan', axis=1, inplace=True)

area=area.drop(['BusinessYear','SourceName', 'ImportDate'], axis = 1)
area.info()


zipcodes=area[area.ZipCodes.isnull() == False]

in_state= area_FL[area_FL.CoverEntireState== 'Yes']
uniques={} 
for col in area_FL.columns: 
    uniques[col]=area_FL[col].unique()
uniques
county_list=uniques['County']

for county in county_list:
    list1=['FL','16842','FLS001','BlueOptions','Yes']
    list1.append(county)
    area_FL = area_FL.append(pd.Series(list1, index=area_FL.columns ), ignore_index=True)
    
    #dropna has to be done post split
area_FL=area_FL.dropna()
area_FL['IssuerId']=area_FL['IssuerId'].astype(int)
area_FL['County']=area_FL['County'].astype(int)
area_FL.drop(['StateCode','ServiceAreaName','CoverEntireState'],axis=1,inplace=True)

area_FL.info()

## FIPS

In [ ]:
#this works

fips= pd.read_csv('../data/zipcodes-county-fips-crosswalk/ZIP-COUNTY-FIPS_2017-06.csv',  dtype='str')
fips['STCOUNTYFP']=fips['STCOUNTYFP'].astype(int)
fips['COUNTYNAME'] = fips['COUNTYNAME'].str.replace(r' County', '')
#fips[['County','County_Name']]=fips[['STCOUNTYFP','COUNTYNAME']]
fips = fips[fips.STATE== 'FL']

#fips.reset_index(inplace=True)

FL_zips =pd.DataFrame()
FL_zips[['zipcode','County','County_Name']] = fips[['ZIP','STCOUNTYFP','COUNTYNAME']]

FL_zips.to_csv('../data/clean_files/zipcodes.csv')

FL_county =pd.DataFrame()
FL_county[['County','County_Name']] = fips[['STCOUNTYFP','COUNTYNAME']]


FL_county.sort_values(by=['County'])
FL_county=FL_county.drop_duplicates(keep='first')
result = pd.merge(area_FL,FL_county, how='left', on=['County', 'County'])
#pd.merge(area_FL, FL_county, how='left', left='County',right='County')

# result.to_csv('../data/clean_files/area_FL.csv')

# result

In [ ]:
area_FL=pd.merge(FL_zips,area_FL,how='left', on='County')
area_FL.to_csv('../data/clean_files/area_FL.csv')

## end area

# Clean up Area

In [ ]:
area=area[area.DentalOnlyPlan != 'Yes']
area.drop('DentalOnlyPlan', axis=1, inplace=True)

area=area.drop(['BusinessYear','SourceName', 'ImportDate'], axis = 1)
area.info()

In [ ]:
# grab states w/ zipcode level plans

zipcodes=area[area.ZipCodes.isnull() == False]


area_FL=area[area.StateCode == 'FL'].copy()
'''
EDA shows FL does not have  plans based on 'PartialCounty', ''ZipCodes', or
PartialCountyJustification' so they are being dropped
'''

area_FL=area_FL.drop(['PartialCounty', 'ZipCodes', 'PartialCountyJustification'], axis = 1)

''' only offer Individual plans so that column is being dropped'''
area_FL=area_FL.drop('MarketCoverage', axis = 1)

# Attr

In [ ]:
attr=pd.read_csv('../data/raw_files/Plan_Attributes_PUF.csv',low_memory=False, encoding ='latin1')
x=attr.columns

# FL does NOT have any attr exclusions but other states do.

array(['No', "When sold off the exchange, MetLife's Dental EHB plans and benefits will meet the stated actuarial value, but the exact plan and benefit design may vary according to the terms of the insurance certificate."], dtype=object)

attr.PlanLevelExclusions.unique()


In [ ]:
attr=attr[attr.DentalOnlyPlan != 1]
attr.drop('DentalOnlyPlan', axis=1, inplace=True)


attr=attr.drop(['BusinessYear','SourceName', 'ImportDate',
                'PlanExpirationDate','PlanEffectiveDate',
                'URLForEnrollmentPayment', 'FormularyURL','IsGuaranteedRate'], axis=1)
# 
attr=attr.drop(['IndianPlanVariationEstimatedAdvancedPaymentAmountPerEnrollee','ChildOnlyPlanId',
                'HSAOrHRAEmployerContribution', 'HSAOrHRAEmployerContributionAmount','PlanLevelExclusions'], axis = 1)

attr=attr.drop(['MEHBInnTier1IndividualMOOP','MEHBInnTier1FamilyPerPersonMOOP',
                'MEHBInnTier1FamilyPerGroupMOOP','MEHBInnTier2IndividualMOOP',
                'MEHBInnTier2FamilyPerPersonMOOP','MEHBInnTier2FamilyPerGroupMOOP',
                'MEHBOutOfNetIndividualMOOP','MEHBOutOfNetFamilyPerPersonMOOP',
                'MEHBOutOfNetFamilyPerGroupMOOP','MEHBCombInnOonIndividualMOOP',
                'MEHBCombInnOonFamilyPerPersonMOOP','MEHBCombInnOonFamilyPerGroupMOOP',
                'DEHBInnTier1IndividualMOOP','DEHBInnTier1FamilyPerPersonMOOP',
                'DEHBInnTier1FamilyPerGroupMOOP','DEHBInnTier2IndividualMOOP',
                'DEHBInnTier2FamilyPerPersonMOOP','DEHBInnTier2FamilyPerGroupMOOP',
                'DEHBOutOfNetIndividualMOOP','DEHBOutOfNetFamilyPerPersonMOOP',
                'DEHBOutOfNetFamilyPerGroupMOOP','DEHBCombInnOonIndividualMOOP',
                'DEHBCombInnOonFamilyPerPersonMOOP','DEHBCombInnOonFamilyPerGroupMOOP',
                'TEHBInnTier1IndividualMOOP','TEHBInnTier1FamilyPerPersonMOOP',
                'TEHBInnTier1FamilyPerGroupMOOP','TEHBInnTier2IndividualMOOP',
                'TEHBInnTier2FamilyPerPersonMOOP','TEHBInnTier2FamilyPerGroupMOOP',
                'TEHBOutOfNetIndividualMOOP','TEHBOutOfNetFamilyPerPersonMOOP',
                'TEHBOutOfNetFamilyPerGroupMOOP','TEHBCombInnOonIndividualMOOP',
                'TEHBCombInnOonFamilyPerPersonMOOP','TEHBCombInnOonFamilyPerGroupMOOP',
                'MEHBDedInnTier1Individual','MEHBDedInnTier1FamilyPerPerson',
                'MEHBDedInnTier1FamilyPerGroup','MEHBDedInnTier1Coinsurance',
                'MEHBDedInnTier2Individual','MEHBDedInnTier2FamilyPerPerson',
                'MEHBDedInnTier2FamilyPerGroup','MEHBDedInnTier2Coinsurance',
                'MEHBDedOutOfNetIndividual','MEHBDedOutOfNetFamilyPerPerson',
                'MEHBDedOutOfNetFamilyPerGroup','MEHBDedCombInnOonIndividual',
                'MEHBDedCombInnOonFamilyPerPerson','MEHBDedCombInnOonFamilyPerGroup',
                'DEHBDedInnTier1Individual','DEHBDedInnTier1FamilyPerPerson',
                'DEHBDedInnTier1FamilyPerGroup','DEHBDedInnTier1Coinsurance',
                'DEHBDedInnTier2Individual','DEHBDedInnTier2FamilyPerPerson',
                'DEHBDedInnTier2FamilyPerGroup','DEHBDedInnTier2Coinsurance',
                'DEHBDedOutOfNetIndividual','DEHBDedOutOfNetFamilyPerPerson',
                'DEHBDedOutOfNetFamilyPerGroup','DEHBDedCombInnOonIndividual',
                'DEHBDedCombInnOonFamilyPerPerson','DEHBDedCombInnOonFamilyPerGroup',
                'TEHBDedInnTier1Individual','TEHBDedInnTier1FamilyPerPerson',
                'TEHBDedInnTier1FamilyPerGroup','TEHBDedInnTier1Coinsurance',
                'TEHBDedInnTier2Individual','TEHBDedInnTier2FamilyPerPerson',
                'TEHBDedInnTier2FamilyPerGroup','TEHBDedInnTier2Coinsurance',
                'TEHBDedOutOfNetIndividual','TEHBDedOutOfNetFamilyPerPerson',
                'TEHBDedOutOfNetFamilyPerGroup','TEHBDedCombInnOonIndividual',
                'TEHBDedCombInnOonFamilyPerPerson','TEHBDedCombInnOonFamilyPerGroup'], axis =1)

#'PlanVariantMarketingName'
attr.drop(['MarketCoverage', 'TIN','HIOSProductId', 'HPID', 'FormularyId', 'IsNewPlan',
       'DesignType', 'UniquePlanDesign', 'QHPNonQHPTypeId',
       'CompositeRatingOffered', 'EHBPercentTotalPremium',
       'EHBPediatricDentalApportionmentQuantity', 'OutOfCountryCoverage',
       'OutOfCountryCoverageDescription', 'OutOfServiceAreaCoverage',
       'OutOfServiceAreaCoverageDescription',
       'CSRVariationType', 'IssuerActuarialValue',
       'AVCalculatorOutputNumber', 'MedicalDrugDeductiblesIntegrated',
       'MedicalDrugMaximumOutofPocketIntegrated', 'MultipleInNetworkTiers',
       'FirstTierUtilization', 'SecondTierUtilization',
       'SpecialtyDrugMaximumCoinsurance', 'InpatientCopaymentMaximumDays',
       'BeginPrimaryCareCostSharingAfterNumberOfVisits',
       'BeginPrimaryCareDeductibleCoinsuranceAfterNumberOfCopays',
       'IsHSAEligible', ],axis=1,inplace=True)


In [ ]:
attr_FL=attr[attr.StateCode == 'FL'].copy()
attr_FL.drop('StateCode',axis=1,inplace=True)
#attr_FL.info()
#len(attr_FL.PlanId.unique())
#attr_FL.columns()
#attr_FL.DiseaseManagementProgramsOffered.unique()
#values = {'A': 0, 'B': 1, 'C': 2, 'D': 3}
attr_FL.fillna('No', inplace=True)
#attr_FL.DiseaseManagementProgramsOffered
attr_FL.info()
# plan ID = 1356, Component = 303
#attr_FL.PlanLevelExclusions.unique()

In [ ]:
#df.replace([0, 1, 2, 3], [4, 3, 2, 1])
attr.replace(['Weight Loss Management Program','High Blood Pressure & High Cholesterol Management Program',],[' Weight Loss Programs',' Blood Pressure-Cholesterol Management Program'],inplace=True)
attr.DiseaseManagementProgramsOffered.unique()

In [ ]:
prevention =pd.concat([attr_FL[['PlanId']], attr_FL['DiseaseManagementProgramsOffered'].str.split(', ', expand=True)], axis=1)
prevention.drop('PlanId', axis=1, inplace=True)
prevention.fillna(0,inplace=True)
prevention.replace(['Weight Loss Management Program','High Blood Pressure & High Cholesterol Management Program'],[' Weight Loss Programs',' Blood Pressure-Cholesterol Management Program'],inplace=True)
prev_dump=prevention.stack().str.get_dummies().sum(level=0)
prev_dump.drop('0',axis=1, inplace=True)
prev_dump.rename(columns={'Pain Management':'Pain','Weight Loss Programs':'Weight Loss'}, inplace=True)
prev_dump.columns = [str(col) + ' Management Program' for col in prev_dump.columns]


prev_dump['PlanId']=attr_FL['PlanId']
#prev_dump.columns
#prev_dump.info()
#new=attr_FL.join(prev_dump, how='left', on='PlanId')
#new=pd.concat([attr_FL, prev_dump], axis=1)
new=attr_FL.set_index('PlanId').join(prev_dump.set_index('PlanId'))
new.reset_index(inplace = True)

#attr_FL
#attr_FL.columns
#attr_FL.merge(prev_dump, on='PlanId')
#export = attr_FL['PlanId',]
#attr_FL.drop(['
#attr_FL
#new.drop('PlanId', inplace=True)
# export=new[['PlanId','PlanMarketingName','PlanType', 'MetalLevel', 'Asthma Management Program',
#        'Depression Management Program', 'Diabetes Management Program',
#        'Heart Disease Management Program',
#        'High Blood Pressure & High Cholesterol Management Program',
#        'Low Back Pain Management Program', 'Pain Management Program',
#        'Pregnancy Management Program', 'Weight Loss Management Program']]
# export
#'IssuerId'
#  'NetworkId','ServiceAreaId'
new.drop(['PlanMarketingName','ChildOnlyOffering',
       'WellnessProgramOffered', 'DiseaseManagementProgramsOffered',
       'NationalNetwork'],axis=1, inplace=True)

new.info()
#len(new.PlanId.unique())

## attr raw

In [ ]:
attr=pd.read_csv('../data/raw_files/Plan_Attributes_PUF.csv',low_memory=False, encoding ='latin1')
#temp=attr[attr['HIOSProductId']=='12379FL001']
#temp

In [ ]:
attr=attr[attr.DentalOnlyPlan != 1]
attr.drop('DentalOnlyPlan', axis=1, inplace=True)
'''
not FL specific
'BusinessYear','SourceName', 'ImportDate','- dropping because no value

'PlanExpirationDate' = 12/31/2020 for all plans
PlanEffectiveDate = 1/1/2020 for all plans
URLForEnrollmentPayment, FormularyURL
IsGuaranteedRate empty for all plans
'''

attr=attr.drop(['BusinessYear','SourceName', 'ImportDate',
                'PlanExpirationDate','PlanEffectiveDate',
                'URLForEnrollmentPayment', 'FormularyURL','IsGuaranteedRate'], axis=1)
'''
FL only (so far) - all exist elsewhere
'MarketCoverage, 'CompositeRatingOffered' only has Individual
'MarketCoverage, 'CompositeRatingOffered' only has Individual
'IndianPlanVariationEstimatedAdvancedPaymentAmountPerEnrollee = $0 for all'
'ChildOnlyPlanId' empty column 
''HSAOrHRAEmployerContribution', 'HSAOrHRAEmployerContributionAmount' - empty column for FL
'PlanLevelExclusions' -only note for FL says if sold not on the exchange, plan is different

'''
attr=attr.drop(['IndianPlanVariationEstimatedAdvancedPaymentAmountPerEnrollee','ChildOnlyPlanId',
                'HSAOrHRAEmployerContribution', 'HSAOrHRAEmployerContributionAmount','PlanLevelExclusions'], axis = 1)
#'CompositeRatingOffered','MarketCoverage'


# States required to report actuarial data of plans not on the exchange. Dropping said plans
#attr=attr[attr.QHPNonQHPTypeId != 'Off Exchange']

#I can't figure out what belongs to what so dropping for now 
attr=attr.drop(['MEHBInnTier1IndividualMOOP','MEHBInnTier1FamilyPerPersonMOOP',
                'MEHBInnTier1FamilyPerGroupMOOP','MEHBInnTier2IndividualMOOP',
                'MEHBInnTier2FamilyPerPersonMOOP','MEHBInnTier2FamilyPerGroupMOOP',
                'MEHBOutOfNetIndividualMOOP','MEHBOutOfNetFamilyPerPersonMOOP',
                'MEHBOutOfNetFamilyPerGroupMOOP','MEHBCombInnOonIndividualMOOP',
                'MEHBCombInnOonFamilyPerPersonMOOP','MEHBCombInnOonFamilyPerGroupMOOP',
                'DEHBInnTier1IndividualMOOP','DEHBInnTier1FamilyPerPersonMOOP',
                'DEHBInnTier1FamilyPerGroupMOOP','DEHBInnTier2IndividualMOOP',
                'DEHBInnTier2FamilyPerPersonMOOP','DEHBInnTier2FamilyPerGroupMOOP',
                'DEHBOutOfNetIndividualMOOP','DEHBOutOfNetFamilyPerPersonMOOP',
                'DEHBOutOfNetFamilyPerGroupMOOP','DEHBCombInnOonIndividualMOOP',
                'DEHBCombInnOonFamilyPerPersonMOOP','DEHBCombInnOonFamilyPerGroupMOOP',
                'TEHBInnTier1IndividualMOOP','TEHBInnTier1FamilyPerPersonMOOP',
                'TEHBInnTier1FamilyPerGroupMOOP','TEHBInnTier2IndividualMOOP',
                'TEHBInnTier2FamilyPerPersonMOOP','TEHBInnTier2FamilyPerGroupMOOP',
                'TEHBOutOfNetIndividualMOOP','TEHBOutOfNetFamilyPerPersonMOOP',
                'TEHBOutOfNetFamilyPerGroupMOOP','TEHBCombInnOonIndividualMOOP',
                'TEHBCombInnOonFamilyPerPersonMOOP','TEHBCombInnOonFamilyPerGroupMOOP',
                'MEHBDedInnTier1Individual','MEHBDedInnTier1FamilyPerPerson',
                'MEHBDedInnTier1FamilyPerGroup','MEHBDedInnTier1Coinsurance',
                'MEHBDedInnTier2Individual','MEHBDedInnTier2FamilyPerPerson',
                'MEHBDedInnTier2FamilyPerGroup','MEHBDedInnTier2Coinsurance',
                'MEHBDedOutOfNetIndividual','MEHBDedOutOfNetFamilyPerPerson',
                'MEHBDedOutOfNetFamilyPerGroup','MEHBDedCombInnOonIndividual',
                'MEHBDedCombInnOonFamilyPerPerson','MEHBDedCombInnOonFamilyPerGroup',
                'DEHBDedInnTier1Individual','DEHBDedInnTier1FamilyPerPerson',
                'DEHBDedInnTier1FamilyPerGroup','DEHBDedInnTier1Coinsurance',
                'DEHBDedInnTier2Individual','DEHBDedInnTier2FamilyPerPerson',
                'DEHBDedInnTier2FamilyPerGroup','DEHBDedInnTier2Coinsurance',
                'DEHBDedOutOfNetIndividual','DEHBDedOutOfNetFamilyPerPerson',
                'DEHBDedOutOfNetFamilyPerGroup','DEHBDedCombInnOonIndividual',
                'DEHBDedCombInnOonFamilyPerPerson','DEHBDedCombInnOonFamilyPerGroup',
                'TEHBDedInnTier1Individual','TEHBDedInnTier1FamilyPerPerson',
                'TEHBDedInnTier1FamilyPerGroup','TEHBDedInnTier1Coinsurance',
                'TEHBDedInnTier2Individual','TEHBDedInnTier2FamilyPerPerson',
                'TEHBDedInnTier2FamilyPerGroup','TEHBDedInnTier2Coinsurance',
                'TEHBDedOutOfNetIndividual','TEHBDedOutOfNetFamilyPerPerson',
                'TEHBDedOutOfNetFamilyPerGroup','TEHBDedCombInnOonIndividual',
                'TEHBDedCombInnOonFamilyPerPerson','TEHBDedCombInnOonFamilyPerGroup'], axis =1)

#not needed/wanted
#'PlanVariantMarketingName'
attr.drop(['MarketCoverage', 'TIN','HIOSProductId', 'HPID', 'FormularyId', 'IsNewPlan',
       'DesignType', 'UniquePlanDesign', 'QHPNonQHPTypeId',
       'CompositeRatingOffered', 'EHBPercentTotalPremium',
       'EHBPediatricDentalApportionmentQuantity', 'OutOfCountryCoverage',
       'OutOfCountryCoverageDescription', 'OutOfServiceAreaCoverage',
       'OutOfServiceAreaCoverageDescription',
       'CSRVariationType', 'IssuerActuarialValue',
       'AVCalculatorOutputNumber', 'MedicalDrugDeductiblesIntegrated',
       'MedicalDrugMaximumOutofPocketIntegrated', 'MultipleInNetworkTiers',
       'FirstTierUtilization', 'SecondTierUtilization',
       'SpecialtyDrugMaximumCoinsurance', 'InpatientCopaymentMaximumDays',
       'BeginPrimaryCareCostSharingAfterNumberOfVisits',
       'BeginPrimaryCareDeductibleCoinsuranceAfterNumberOfCopays',
       'IsHSAEligible', ],axis=1,inplace=Tr

In [ ]:
ronnters=attr[['IssuerId', 'StandardComponentId', 'PlanMarketingName', 'NetworkId',
       'ServiceAreaId','PlanId']]

special=attr[['IsNoticeRequiredForPregnancy','IsReferralRequiredForSpecialist','SpecialistRequiringReferral']]
attr.drop(['IsNoticeRequiredForPregnancy','IsReferralRequiredForSpecialist','SpecialistRequiringReferral'], axis=1, inplace=True)

ld_diab_fx_mon=attr[['SBCHavingaBabyDeductible',
       'SBCHavingaBabyCopayment', 'SBCHavingaBabyCoinsurance',
       'SBCHavingaBabyLimit', 'SBCHavingDiabetesDeductible',
       'SBCHavingDiabetesCopayment', 'SBCHavingDiabetesCoinsurance',
       'SBCHavingDiabetesLimit', 'SBCHavingSimplefractureDeductible',
       'SBCHavingSimplefractureCopayment',
       'SBCHavingSimplefractureCoinsurance', 'SBCHavingSimplefractureLimit']]
attr.drop(['SBCHavingaBabyDeductible',
       'SBCHavingaBabyCopayment', 'SBCHavingaBabyCoinsurance',
       'SBCHavingaBabyLimit', 'SBCHavingDiabetesDeductible',
       'SBCHavingDiabetesCopayment', 'SBCHavingDiabetesCoinsurance',
       'SBCHavingDiabetesLimit', 'SBCHavingSimplefractureDeductible',
       'SBCHavingSimplefractureCopayment',
       'SBCHavingSimplefractureCoinsurance', 'SBCHavingSimplefractureLimit'],axis=1,inplace=True)

plan_docs=[['URLForSummaryofBenefitsCoverage','PlanBrochure']]
attr.drop(['URLForSummaryofBenefitsCoverage','PlanBrochure'],axis=1,inplace=True)


In [ ]:
attr_FL=attr[attr.StateCode == 'FL'].copy()
attr_FL.drop('StateCode',axis=1,inplace=True)
attr_FL.info()
len(attr_FL.StandardComponentId.unique())
#attr_FL.columns()

In [ ]:
uniques={} 
for col in attr_FL.columns: 
    uniques[col]=attr_FL[col].unique()
#attr_FL.NationalNetwork.unique()
uniques

# end attr

# eigenvalues

In [ ]:
# sort eigenvalue in decreasing order
idx = np.argsort(match.values)[0]

plans = names[idx]
    # sort eigenvectors according to same indeplqnw
plans[946:]
match[idx]
np.unique(match.values,return_counts=True)

# rejections

In [ ]:
rejections = pd.read_csv('../data/quality_dbs/rejection_rates.csv')
rejections.rename(columns={'Is_Issuer_New_to_Exchange? (Yes_or_No)':'new_plan'},inplace=True )

rejections=rejections.replace('#DIV/0!','0')

rejections.Plan_denial_rate=pd.to_numeric(rejections.Plan_denial_rate)

rejections

rejections['IssuerEnroll']=rejections.groupby('Issuer_ID')['Enrollment_Data'].transform(np.sum)
rejections['I_Disenroll']=rejections.groupby('Issuer_ID')['Disenrollment_Data'].transform(np.sum)
rejections['Issuer_Disenroll']=rejections['I_Disenroll']/rejections['IssuerEnroll']*100
rejections


# def perc(df_name,col_to_change,base_col,new_value):
#     for col in col_to_change:
#         df_name[col]=df_name[col].round(2)
#         df_name[col] = np.where((df_name[base_col]== 1),new_value,df_name[col])

# x=['Issuer_Denial_rate','Plan_denial_rate','Disenrollment_Rate']
# perc('rejections',x,'new_plan','New Plan')     
rejections.Issuer_Denial_rate=rejections.Issuer_Denial_rate.round(1)
rejections.Plan_denial_rate=rejections.Plan_denial_rate.round(1)
rejections.Disenrollment_Rate=rejections.Disenrollment_Rate.round(1)
rejections.Issuer_Disenroll=rejections.Issuer_Disenroll.round(1)
rejections['Plan_denial_rate'] = np.where((rejections['new_plan']=='Yes'),'New Plan',rejections['Plan_denial_rate'])
rejections['Issuer_Disenroll'] = np.where((rejections['new_plan']=='Yes'),'New Plan',rejections['Issuer_Disenroll'])
rejections['Issuer_Denial_rate'] = np.where((rejections['new_plan']=='Yes'),'New Plan',rejections['Issuer_Denial_rate'])
rejections.Disenrollment_Rate=rejections.Disenrollment_Rate.replace('nan','Disenrollment not Provided')
rejections.Issuer_Disenroll=rejections.Issuer_Disenroll.replace('nan','Disenrollment not Provided')
rejections=rejections.replace('nan','Denials not Provided')
print(rejections.columns)
# rejections.drop(['Issuer_Name', 'Issuer_ID', 'new_plan',
#        'Issuer_Claims_Received', 'Issuer_Claims_Denials',
#        'Plan_Number_Claims_Received', 'Plan_Number_Claims_Denied',
#        'Enrollment_Data', 'Disenrollment_Data'], axis=1,inplace=True)
rejections.rename(columns ={'Plan_ID':'StandardComponentId'},inplace=True)
# 'Issuer_Name', 'Issuer_ID', 'new_plan', 'Plan_ID',
#        'Issuer_Claims_Received', 'Issuer_Claims_Denials', 'Issuer_Denial_rate',
#        'Plan_Number_Claims_Received', 'Plan_Number_Claims_Denied',
#        'Plan_denial_rate', 'Enrollment_Data', 'Disenrollment_Data',
#        'Disenrollment_Rate'],


rejections.sort_values('StandardComponentId')
rejections=rejections.drop_duplicates('Issuer_ID')
rejections=rejections.rename(columns={'Issuer_ID':'IssuerId'})

disenroll_rates=rejections[['IssuerId','Issuer_Denial_rate','Issuer_Disenroll']]
disenroll_rates.to_csv('../data/clean_files/clean_files/rejection_rates')

approval_scores=pd.read_csv('../data/quality_dbs/Nationwide_QRS_PUF_PY2020.csv')


## from attr cleam

In [ ]:
rejections = pd.read_csv('../data/quality_dbs/rejection_rates.csv')
rejections.rename(columns={'Is_Issuer_New_to_Exchange? (Yes_or_No)':'new_plan'},inplace=True )

rejections=rejections.replace('#DIV/0!','0')

rejections.Plan_denial_rate=pd.to_numeric(rejections.Plan_denial_rate)

len(rejections.Plan_ID.unique())
rejections.info()


rejections.Issuer_Denial_rate=rejections.Issuer_Denial_rate.round(2)
rejections.Plan_denial_rate=rejections.Plan_denial_rate.round(2)
rejections.Disenrollment_Rate=rejections.Disenrollment_Rate.round(2)
rejections['Plan_denial_rate'] = np.where((rejections['new_plan']==1),'New Plan',rejections['Plan_denial_rate'])
rejections['Disenrollment_Rate'] = np.where((rejections['new_plan']==1),'New Plan',rejections['Disenrollment_Rate'])
rejections['Issuer_Denial_rate'] = np.where((rejections['new_plan']==1),'New Plan',rejections['Issuer_Denial_rate'])
rejections.Disenrollment_Rate=rejections.Disenrollment_Rate.replace('nan','Disenrollment not Provided')
rejections=rejections.replace('nan','Denials not Provided')


# rejections_Issuer=rejections[['Issuer_ID','Issuer_Denial_rate','Disenrollment_Rate']]
# denials = rejections.groupby('Issuer_ID')['Issuer_Denial_rate'].max()
# issuer_level=pd.DataFrame()
# issuer_level['Issuer_Denial_rate']=rejections.groupby('Issuer_ID')['Issuer_Denial_rate'].max()
# issuer_level['Disenrollment_Rate']=rejections.groupby('Issuer_ID')['Disenrollment_Rate'].max()
# issuer_level.reset_index(inplace=True)
# # issuer_level.rename('Issuer_ID',)


# rejections['Issuer_Denial_rate'] = np.where((rejections['Issuer_ID']==issuer_level['Issuer_ID']),issuer_level['Issuer_Denial_rate'],rejections['Issuer_Denial_rate'])
# 'Issuer_ID','Issuer_Name'
rejections.drop(['new_plan','Issuer_Claims_Received', 'Issuer_Claims_Denials',
       'Plan_Number_Claims_Received', 'Plan_Number_Claims_Denied',
       'Enrollment_Data', 'Disenrollment_Data'], axis=1,inplace=True)
rejections.rename(columns ={'Plan_ID':'StandardComponentId'},inplace=True)

rejections.info()

## from attr raw

In [ ]:
rejections = pd.read_csv('../data/quality_dbs/rejection_rates.csv')
rejections.rename(columns={'Is_Issuer_New_to_Exchange? (Yes_or_No)':'new_plan'},inplace=True )

rejections=rejections.replace('#DIV/0!','0')

rejections.Plan_denial_rate=pd.to_numeric(rejections.Plan_denial_rate)

rejections


rejections_Issuer=rejections[['Issuer_ID','Issuer_Denial_rate','Disenrollment_Rate']]
rejections_Issuer.info()
denials = rejections.groupby('Issuer_ID')['Issuer_Denial_rate'].max()
issuer_level=pd.DataFrame()
issuer_level["Plan_Deny_Rate-C"]=rejections.groupby('Issuer_ID')['Issuer_Denial_rate'].max()
issuer_level["DisEnroll-C"]=rejections.groupby('Issuer_ID')['Disenrollment_Rate'].max()
issuer_level


# def perc(df_name,col_to_change,base_col,new_value):
#     for col in col_to_change:
#         df_name[col]=df_name[col].round(2)
#         df_name[col] = np.where((df_name[base_col]== 1),new_value,df_name[col])

# x=['Issuer_Denial_rate','Plan_denial_rate','Disenrollment_Rate']
# perc('rejections',x,'new_plan','New Plan')     
rejections.Issuer_Denial_rate=rejections.Issuer_Denial_rate.round(2)
rejections.Plan_denial_rate=rejections.Plan_denial_rate.round(2)
rejections.Disenrollment_Rate=rejections.Disenrollment_Rate.round(2)
rejections['Plan_denial_rate'] = np.where((rejections['new_plan']==1),'New Plan',rejections['Plan_denial_rate'])
rejections['Disenrollment_Rate'] = np.where((rejections['new_plan']==1),'New Plan',rejections['Disenrollment_Rate'])
rejections['Issuer_Denial_rate'] = np.where((rejections['new_plan']==1),'New Plan',rejections['Issuer_Denial_rate'])
rejections.Disenrollment_Rate=rejections.Disenrollment_Rate.replace('nan','Disenrollment not Provided')
rejections=rejections.replace('nan','Denials not Provided')
print(rejections.columns)
rejections.drop(['Issuer_Name', 'Issuer_ID', 'new_plan',
       'Issuer_Claims_Received', 'Issuer_Claims_Denials',
       'Plan_Number_Claims_Received', 'Plan_Number_Claims_Denied',
       'Enrollment_Data', 'Disenrollment_Data'], axis=1,inplace=True)
rejections.rename(columns ={'Plan_ID':'StandardComponentId'},inplace=True)
# 'Issuer_Name', 'Issuer_ID', 'new_plan', 'Plan_ID',
#        'Issuer_Claims_Received', 'Issuer_Claims_Denials', 'Issuer_Denial_rate',
#        'Plan_Number_Claims_Received', 'Plan_Number_Claims_Denied',
#        'Plan_denial_rate', 'Enrollment_Data', 'Disenrollment_Data',
#        'Disenrollment_Rate'],


rejections.sort_values('StandardComponentId')

## end rejections

#  Benefits

# Benefits_cost

In [ ]:
pre_pivot = ben_cost[['PlanId','BenefitName', 'IsCovered']]
post_pivot= pre_pivot.pivot(index='PlanId', columns='BenefitName', values='IsCovered')
post_pivot.head()

pivot_list=[]
for col in post_pivot:
    pivot_list.append(col)
pivot_list[]

#df.pivot(index='foo', columns='bar', values='baz')




ben_FL=ben_cost[ben_cost.StateCode == 'FL'].copy()
ben_FL.info()


not_covered = ben_FL[ben_FL.IsCovered == 'Not Covered']

## benefits clean-uo

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_rows',1500)
pd.set_option('display.max_columns', 500)

In [ ]:
#area=pd.read_csv('../data/clean_files/area_FL.csv')
ben_cost=pd.read_csv('../data/raw_files/Benefits_Cost_Sharing_PUF.csv',low_memory=False, encoding ='latin1')

ben_cost=ben_cost.drop(['BusinessYear','SourceName', 'ImportDate'], axis = 1)
ben_cost=ben_cost[ben_cost.StateCode == 'FL'].copy()

ben=ben_cost.copy()
ben.drop(columns=['CopayInnTier1', 'CopayInnTier2', 'CopayOutofNet', 'CoinsInnTier1','CoinsInnTier2', 'CoinsOutofNet', 'IsEHB',
                'QuantLimitOnSvc', 'LimitQty', 'LimitUnit', 'Explanation','EHBVarReason', 'IsExclFromInnMOOP', 'IsExclFromOonMOOP'], axis=1, inplace=True)

dental = ['Basic Dental Care - Child', 'Orthodontia - Child', 'Major Dental Care - Child',
          'Basic Dental Care - Adult', 'Orthodontia - Adult', 'Major Dental Care - Adult',
          'Dental Check-Up for Children', 'Routine Dental Services (Adult)', 'Accidental Dental',
          'Dental Anesthesia','Congenital Anomaly, including Cleft Lip/Palate', 'Dental X-rays',
          'Topical Flouride', 'Sealants', 'Fillings', 'Recementation of Space Maintainers', 
          'Removal of Fixed Space Maintainers', 'Restorative Services', 
          'Periodontal Root Scaling and Planing', 'Periodontal Maintenance', 'Periodontal and Osseous Surgery',
          'Occlusal Adjustments', 'Root Canal Therapy and Retreatment', 'Periradicular Surgical Procedures',
          'Partial Pulpotomy', 'Vital Pulpotomy', 'Denture Adjustments',
          'Initial Placement of Bridges and Dentures', 'Tissue Conditioning', 'Reline and Rebase', 
          'Post and Core Build-up', 'Extractions', 'Complex Oral Surgery', 'Implants',
          'Immediate Dentures', 'Anesthesia Services for Dental Care', 'Accidental Dental Adult','Denture Reline and Rebase']

drugs = ['Off Label Prescription Drugs', 'Generic Drugs', 'Preferred Brand Drugs','Non-Preferred Brand Drugs','Specialty Drugs',
         'Tier 2 Generic Drugs', 'Preferred Generic Drugs']
ben=ben[ben['BenefitName'].isin(dental) == False]
ben=ben[ben['BenefitName'].isin(drugs) == False]
#[df[df["column"].isin(["value"]) == False]]
#ben.drop('StateCode', axis=1, inplace=True)

'''
add these if needed\\['IssuerId', 'StandardComponentId', 'PlanId', 'BenefitName',
'IsCovered','QuantLimitOnSvc', 'LimitQty', 'LimitUnit', 'Exclusions']
'StateCode','CopayInnTier1', 'CopayInnTier2', 'CopayOutofNet', 'CoinsInnTier1',
       'CoinsInnTier2', 'CoinsOutofNet', 'IsEHB',
       'QuantLimitOnSvc', 'LimitQty', 'LimitUnit', 'Explanation',
       'EHBVarReason', 'IsExclFromInnMOOP', 'IsExclFromOonMOOP'
'''

ben.info()

### Ben clean up FL

In [ ]:
ben_FL=ben[ben['StateCode']=='FL']

ben.drop('StateCode', axis=1, inplace=True)

cover_FL = ben_FL[ben_FL['IsCovered']== 'Covered']
not_cover_FL = ben[ben['IsCovered']== 'Not Covered']
cover_FL.head()

#### FL covered

In [ ]:
df1=cover_FL[['StandardComponentId','PlanId','BenefitName']]
#ben_dummies=pd.get_dummies(df1.BenefitName)
#df1=pd.concat([df1,ben_dummies], axis=1)
# df2=pd.pivot_table(df1, values ='BenefitName')

#pd.pivot_table(df1, values="value", index=["city"], columns="cuisine", fill_value=0)
df1['value']=1
table = pd.pivot_table(df1, values=['StandardComponentId'], index=['PlanId'],columns=['BenefitName'],aggfunc='count')
#table = pd.pivot_table(df1, values=['value'], index=['PlanId'],columns=['BenefitName'],fill_value=0)

#table.columns = table.columns.droplevel()
#table.to_csv('test.csv')
table.info()


#### overed Exclusions

In [ ]:
conditions = cover_FL[['PlanId','StandardComponentId','BenefitName', 'Exclusions']]
 #DataFrame.pivot(self, index=None, columns=None, values=None)[source]
conditions = conditions[conditions.Exclusions.notna()]
unies= conditions.drop_duplicates()
unies.sort_values('BenefitName')
#unies.BenefitName.unique()
#pandas.pivot_table(data, values=None, index=None, columns=None, aggfunc='mean', fill_value=None, margins=False, dropna=True, margins_name='All', observed=False)
#conditionals=conditions.pivot(values='Exclusions',columns='BenefitName',index='PlanId')
#conditionals.info()


conditional_covered =['Delivery and All Inpatient Services for Maternity Care','Diabetes Care Management','Mental/Behavioral Health Outpatient Services','Mental/Behavioral Health Inpatient Services','Substance Abuse Disorder Outpatient Services','Substance Abuse Disorder Inpatient Services','Durable Medical Equipment']
conditionals=unies.loc[unies['BenefitName'].isin(conditional_covered)]
conditionals.to_csv('../data/clean_files/covered_exclusions.csv')

#flattened = pd.DataFrame(pivoted.to_records())
df3=pd.DataFrame(table.to_records())

old_names=df3.columns
old_names=old_names.drop(['PlanId'])
new_names=[x[1] for x in old_names]
old_names


exclusions_dict = {'Delivery and All Inpatient Services for Maternity Care': ['Pre-certification/pre-authorization of coverage required for non-emergency admissions','Pre-certification/pre-authorization of coverage required for non-emergency admissions.',
                                                                         ' Cost sharing is a per day limit.'],
              'Diabetes Care Management':['Available at FHCP Pharmacy Only. Mail Order not available',
                                          'In order for services to be covered, diabetes outpatient self-management training and educational services must be provided under the direct supervision of an FHCP certified Diabetes Educator or board-certified Physician spcializing in endocrinology. Additionally, nutrition counseling must be provided by a licensed FHCP contracted dietitian.'],
              'Durable Medical Equipment':'Items that are primarily for convenience or comfort and items available over-the-counter are excluded. The replacement of equipment is also excluded, unless it is non-functional and not practically repairable.',
              'Mental/Behavioral Health Inpatient Services': 'Any inpatient Services provided in a residential treatment Facility are excluded from Coverage',
              'Mental/Behavioral Health Outpatient Services':'Any outpatient Services provided in a residential treatment Facility are excluded from Coverage',
              'Substance Abuse Disorder Inpatient Services':'Any inpatient Services provided in a residential treatment Facility are excluded from Coverage.',
              'Substance Abuse Disorder Outpatient Services':'Any outpatient Services provided in a residential treatment Facility are excluded from Coverage.'
             }

## Benefits_cost

In [ ]:
not_covered_knockout = ['Treatment for Temporomandibular Joint Disorders','Infertility Treatment',
                        'Routine Eye Exam (Adult)','Prosthetic Devices','Infusion Therapy','Allergy Testing']
l_d=['Newborn Services Other', 'Abortion for Which Public Funding is Prohibited']
junk=['Prescription Drugs Other']                       
diabetes = ['Diabetes Care Management']                    ]
weight_loss = ['Bariatric Surgery','Weight Loss Programs','Nutritional Counseling',] 
feet = ['Routine Foot Care']
 alt_med=['Acupuncture','Chiropractic Care']                    
hearing = ['Hearing Aids','Newborn Hearing Screening']                      
conditional = {'Private-Duty Nursing','Cosmetic Surgery'}
nursing =['Long-Term/Custodial Nursing Home Care','Private-Duty Nursing','Skilled Nursing Facility']
mental_health = 'Mental Health Other',
conditional = {'Private-Duty Nursing':'Not covered unless medically necessary.'
              'Cosmetic Surgery':'Due to accident, congenital defect, prosthesis removal, or mastectomy'
               'Abortion for Which Public Funding is Prohibited': 'life of mother',
               'Nutritional Counseling': 'Nutritional counselling needed for Diabetes or Maternity'

               
covered = ben_cost[ben_cost.IsCovered == 'Covered']
covered.info()     
               
               covered_uniques={} 
for col in covered.columns: 
    covered_uniques[col]=covered[col].unique()
covered_uniques

#len(uniques['Exclusions'])
for key in covered_uniques:
    print(key,' ',len(covered_uniques[key]))
               
               

## Exclusions for covered

In [ ]:
injury = ['Primary Care Visit to Treat an Injury or Illness']
specialist= ['Specialist Visit']
alt_med =['Other Practitioner Office Visit (Nurse, Physician Assistant)']
outpatient_site = ['Outpatient Facility Fee (e.g., Ambulatory Surgery Center)',]

cross_table =covered[covered.BenefitName == 'Outpatient Facility Fee (e.g., Ambulatory Surgery Center)' ]
cross_table.groupby('Exclusions').count()

#### list of exclusions national


'Hospice Services', 'Urgent Care Centers or Facilities',
       'Home Health Care Services', 'Emergency Room Services',
       'Emergency Transportation/Ambulance',
       'Inpatient Hospital Services (e.g., Hospital Stay)',
       'Inpatient Physician and Surgical Services',
       'Skilled Nursing Facility', 'Prenatal and Postnatal Care',
       'Delivery and All Inpatient Services for Maternity Care',
       'Mental/Behavioral Health Outpatient Services',
       'Mental/Behavioral Health Inpatient Services',
       'Substance Abuse Disorder Outpatient Services',
       'Substance Abuse Disorder Inpatient Services', 'Generic Drugs',
       'Preferred Brand Drugs', 'Non-Preferred Brand Drugs',
       'Specialty Drugs', 'Outpatient Rehabilitation Services',
       'Habilitation Services', 'Chiropractic Care',
       'Durable Medical Equipment', 'Imaging (CT/PET Scans, MRIs)',
       'Preventive Care/Screening/Immunization', 'Routine Foot Care',
       'Acupuncture', 'Routine Eye Exam for Children',
       'Eye Glasses for Children', 'Rehabilitative Speech Therapy',
       'Rehabilitative Occupational and Rehabilitative Physical Therapy',
       'Well Baby Visits and Care',
       'Laboratory Outpatient and Professional Services',
       'X-rays and Diagnostic Imaging',
       'Abortion for Which Public Funding is Prohibited', 'Transplant',
       'Accidental Dental', 'Dialysis', 'Allergy Testing', 'Chemotherapy',
       'Radiation', 'Diabetes Education', 'Prosthetic Devices',
       'Infusion Therapy', 'Nutritional Counseling',
       'Reconstructive Surgery', 'Routine Eye Exam (Adult)',
       'Hearing Aids', 'Topical Flouride', 'Fillings', 'Sealants',
       'Recementation of Space Maintainers',
       'Removal of Fixed Space Maintainers', 'Restorative Services',
       'Periodontal Root Scaling and Planing', 'Periodontal Maintenance',
       'Periodontal and Osseous Surgery', 'Occlusal Adjustments',
       'Root Canal Therapy and Retreatment',
       'Periradicular Surgical Procedures', 'Partial Pulpotomy',
       'Vital Pulpotomy', 'Denture Adjustments',
       'Initial Placement of Bridges and Dentures', 'Tissue Conditioning',
       'Denture Reline and Rebase', 'Post and Core Build-up',
       'Extractions', 'Complex Oral Surgery', 'Implants',
       'Immediate Dentures', 'Infertility Treatment', 'Bariatric Surgery',
       'Major Dental care - Adult HIGH Plan', 'Dental X-rays',
       'Treatment for Temporomandibular Joint Disorders',
       'Private-Duty Nursing', 'Major Dental Care - Adult High Plan',
       'Accidental Dental Adult', 'Wigs', 'Clinical Trials',
       'Diabetes Care Management', 'Inherited Metabolic Disorder - PKU',
       'Off Label Prescription Drugs', 'Prescription Drugs Other',
       'Adult Vision Frames or Lenses', 'Tier 2 Generic Drugs',
       'Dental Anesthesia',
       'Congenital Anomaly, including Cleft Lip/Palate',
       'Orthodontia - Adult', 'Bone Marrow Transplant',
       'Nutrition/Formulas', 'Osteoporosis', 'Eye Glasses for Adults',
       'Anesthesia Services for Dental Care',
       'Enteral/Parenteral and Oral Nutrition Therapy',
       'Osteoporosis Treatment', 'Cardiac and Pulmonary Rehabilitation',
       'Preferred Generic Drugs', 'Genetic Testing Lab Services',
       'Hyperbaric Oxygen Therapy', 'Outpatient Observation',
       'Partial Hospitalization', 'Allergy Injections',
       'Fitness Center Membership', 'Mental Health Office Visit',
       'Substance Abuse Office Visit', 'Telehealth', 'Gym Access',
       'Cosmetic Surgery', 'Weight Loss Programs', 'Well Child Care',
       'Heart Transplant', 'Cosmetic Orthodontia',
       'Non-Preferred Specialty Drugs', 'Autism Spectrum Disorders',
       'Eyeglasses For Adults', 'Eyeglasses for Adults',
       'Non-Emergency Care When Traveling Outside the U.S.',
       'Attention Deficit Disorder', 'Accidental Dental - Child',
       'Bone Marrow Testing', 'Newborn Hearing Screening',
       'Applied Behavior Analysis Based Therapies', 'Topical Fluoride',
       'Major Restorative Services', 'Complete and Partial Dentures',
       'Cardiac Rehabilitation', 'Mental Health Other',
       'Inpatient Rehabilitation',
       'Non-Emergency Care When Traveling Outside of the U.S.',
       'Tier 2 Rx', 'Tier 6 Rx',
       'Prenatal and Postnatal Care - Non-Global Maternity',
       'Post-Mastectomy Care', 'Brain Injury', 'Pediatric Services Other',
       'Transplant Donor Coverage', 'Habilitation - Autism',
       'Major Dental Care - Adult, High Plan',
       'Removal of Fixed Space Miantainers', 'Mammography',
       'Cardiovascular Disease', 'Newborn Services Other',
       'Primary Care E-Visit', 'Post-cochlear implant aural therapy',
       'Ostomy Supplies',
       'Autism Spectrum Disorder - Intensive Level Services',
       'Autism Spectrum Disorder - Non-Intensive Level Services',
       'Post-cochlear implant and aural therapy',
       'Post-cochlear aural therapy', 'Massage Therapy',
       'Complementary Medicine',
       'Post-Cochlear Implant Aural Rehabilitation Therapy',
       'Vision Rehabilitation Therapy', 'Massage', 'Preventative Drugs',
       'Gastric Electrical Stimulation', 'Cochlear Implants',
       'Craniofacial Surgery', 'Specialty Drugs Tier 2',
       'Preventive Drugs',
       'Orthodontic Services for the Treatment of Orofacial Anomalies',
       'Gender Identity Services',
       'Applied Behavior Analysis Based Therapies\t',
       'Autism Spectrum Disorders\t',
       'Orthodontic Services for the Treatment of Orofacial Anomalies\t',
       'Telehealth\t', 'Gender Identity Services \t',
       'Orthodontic Services for the Treatment of Orofacial Anomalies \t',
       'Telehealth \t',
       'Orthodontic Services for the Treatment of OrofacialAnomalies\t',
       'Gender Identity Services\t', 'Generic Drugs Other',
       'Specialty Laboratory Services', 'Testing Services',
       'Active & Fit', 'Adult Optical (hardware)',
       'Corrective Orthodontia to Age 19', 'Posterior Composites',
       'Posterior Composites - Child', 'Implants - Adult',
       'Implants - Child', 'Posterior Composites - Adult',
       'Posterior Composite - Child', 'Posterior Composite - Adult',
       'Naprapathic Service', 'Organ Transplants', 'Bones/Joints',
       'Breast Implant Removal', 'Multiple Sclerosis',
       'Major Dental Care - Adult High Plan \t', 'Dental X-rays\t',
       'Topical Flouride \t', 'Sealants \t',
       'Recementation of Space Maintainers\t',
       'Removal of Fixed Space Maintainers\t', 'Fillings\t',
       'Periodontal Root Scaling and Planing\t',
       'Periodontal Maintenance\t', 'Periodontal and Osseous Surgery\t',
       'Occlusal Adjustments\t', 'Root Canal Therapy and Retreatment\t',
       'Periradicular Surgical Procedures\t', 'Partial Pulpotomy\t',
       'Vital Pulpotomy\t', 'Denture Adjustments\t',
       'Initial Placement of Bridges and Dentures\t',
       'Tissue Conditioning \t', 'Denture Reline and Rebase \t',
       'Post and Core Build-up\t', 'Extractions \t',
       'Complex Oral Surgery\t', 'Immediate Dentures \t', 'E-Visit',
       'Dental Anesthesia - Child', 'Tier 2-Generic Drugs',
       'Adult Frames and Lenses', 'Recementation of Space Maintainers \t',
       'Removal of Fixed Space Maintainers \t', 'Fillings \t',
       'Occlusal Adjustments \t',
       'Periodontal Root Scaling and Planing \t',
       'Periodontal and Osseous Surgery \t',
       'Periradicular Surgical Procedures \t', 'Partial Pulpotomy \t',
       'Vital Pulpotomy \t', 'Complete and Partial Dentures \t',
       'Denture Adjustments \t', 'Second Opinion',
       'Contraceptive Services', 'Early Intervention Services',
       'Infant Formulas', 'Human Leukocyte Antigen Testing',
       'Anesthetics', 'Blood and Blood Services',
       'Inpatient Rehabilitation Services', 'Retail Health Clinics',
       'Low Protein Foods', 'Non Preferred Specialty Drugs',
       'Telemedicine Services', 'Mental Health - Intermediate',
       'Substance Use Disorders - Intermediate',
       'Gender Reassignment Treatment', 'Removal of Space Maintainers\t',
       'Root Canal Therapy and Retreatment \t',
       'Initial Placement of Bridges and Dentures \t',
       'Denture Reline and Rebase\t', 'Complex Oral Surgery \t',
       'Rehabilitative Devices', 'Habilitative Devices',
       'Non Preferred Generic Drugs', 'Non Preferred Speciality Drug',
       'Convenience Care Clinic', 'Telemedicine', 'Topical Flouride\t',
       'Sealants\t', 'Extractions\t', 'Implants\t',
       'Chronic Pain Treatment'], dtype=object)

# End Benefits


# Network

In [ ]:
network=pd.read_csv('../data/raw_files/Network_PUF.csv',low_memory=False, encoding ='latin1')
#businessyear and issuerID only numerical vals
network.head()

#network['DentalOnlyPlan'].unique()


#this gets rid of the Dental Only Plans leaving medical insurance plans then drops the column as superfilous
network=network[network.DentalOnlyPlan != 'Yes']
network=network.drop('DentalOnlyPlan', axis=1)

network=network.drop(['BusinessYear','SourceName', 'ImportDate','NetworkURL'], axis = 1)
network.info()


network_FL=network[network.StateCode == 'FL'].copy()
network_FL.drop(['StateCode','MarketCoverage'],axis=1, inplace=True)
network_FL.columns
network_FL=network_FL[['IssuerId','NetworkId','NetworkName']]


# don't know where this came from - found in simplified

In [ ]:
def find_attributes():
    attributes = pd.read_csv('../data/clean_files/attr_FL.csv')
    attributes=attributes.drop_duplicates('IssuerId')
    #grabbing network names here instead of importing network_FL.csv
    plan_names=attributes[['IssuerId', 'Issuer_Name']]
    attributes.drop(['PlanId','Unnamed: 0','Issuer_ID','Issuer_Denial_rate','Plan_denial_rate',
    'Disenrollment_Rate','Issuer_Denial_rate','Plan_denial_rate','Disenrollment_Rate','StandardComponentId','NetworkId',
    'ServiceAreaId','PlanType','MetalLevel','PlanVariantMarketingName','Issuer_Name','No Management Program'], axis=1, inplace=True)
   
    return attributes, plan_names

attributes, plan_names = find_attributes()

# End Network

#  Quality

In [ ]:
quality=pd.read_csv('../data/quality_dbs/Nationwide_QRS_PUF_PY2020.csv')
quality.columns

df = quality[['IssuerID', 'State', 'Global_rate']]
df=df[df.Global_rate !='NG']

df['Global_rate']=df['Global_rate'].astype(int)



quality.replace(['NG','CSR-I'],[-1,-1],inplace=True)
col_list=quality.columns[2:]

quality.drop(col_list[2], axis=1, inplace=True)

quality['Global_rate']=quality['Global_rate'].astype(int)

uality.replace(['NG','CSR-I'],[-1,-1],inplace=True) 
quality['Global_rate']=quality['Global_rate'].astype(int)

check_plan_nationally=quality[quality['IssuerID']== 48121]
chek_plan_nationally

FL=quality[quality['State']== 'FL']


# End quality